In [ ]:
## 네이버 카페에서 '과외', '과외추천', '과외추천해주세요' 등 검색 
## 지역은 카페명에서 추출해서 "지역별 과외수요"를 알아볼 수 있음 

## https://section.cafe.naver.com/ca-fe/home/search/articles?q=%EA%B3%BC%EC%99%B8%EC%B6%94%EC%B2%9C&p=3

In [96]:
import requests
import json
import pandas as pd 

import os
import sys

from urllib.parse import quote

import re

## Test1..전체 카페에서 "과외추천해주세요" 검색하기
### 문제 : 카페명으로 지역을 어떻게 구분할 것인가? 

In [65]:
client_id = "vD02ixj_z1lpFZ41CeO1"
client_secret = "qk2B9tKmJK"

def cafe_call(keyword,start):
    encText = quote(keyword)
    url = "https://openapi.naver.com/v1/search/cafearticle?query=" + encText+"&sort=date&display=100&start="+str(start) # json 결과
    '''
    요청변수 옵션
    - display : 검색결과 출력건수 (10,100)
    - start : 검색시작위치 (1,1000)
    - sort : 정렬옵션 (sim:유사도순, date:날짜순)
    '''
    result = requests.get(url=url, headers={"X-Naver-Client-Id":client_id,"X-Naver-Client-Secret":client_secret})
    
    if result.status_code == 200:
        #data = json.loads(result.text)
        return result.json()
    else:
        print("error",result.status_code)
    
    '''
    출력결과 옵션
    - lastBuildDate : 검색결과를 생성한 시간 
    - total : 검색결과 문서의 총 개수 (ex : 강남구 과외추천 검색시 2,587)
    - start : 결과문서 중 문서의 시작점-> 한페이지에 100개씩 보여주니까 전체결과가 아님
    - display : 현재 검색결과로 보여준 결과개수 -> items dict안에 들어가는 최대
    - items : 개별검색결과
        - title : 카페 게시글 제목 -> 태그처리필요
        - link : 게시글 링크
        - description : 카페글 요약 -> 태그처리필요
        - cafename : 카페명
        - cafeurl : 카페링크
    
    # 4년치
    
    모든 문서 검색하려고 cafe_call 반복하기?
    # 출처 https://john-analyst.medium.com/%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9D%98-%ED%8A%B9%EC%A0%95-%ED%82%A4%EC%9B%8C%EB%93%9C-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-2c4644001144
    '''

In [68]:
#1000개의 키워드 검색 결과를 받아오기
def get1000results(keyword):
    list = []
    for num in range(0,10):
        list = list + cafe_call(keyword, num * 100 + 1)['items'] # list 안에 키값이 ’item’인 애들만 넣기
    return list

In [89]:
#키워드로 검색한결과
def getDataFrame():
    list = []
    result1 = get1000results("과외추천해주세요")
    result2 = get1000results("과외추천받아요")

    list = list + result1+result2
    
    df = pd.DataFrame.from_records(list)
    df.to_csv("./cafe_data.csv",encoding = 'utf-8-sig')
    
    return df 

In [91]:
# main
df = getDataFrame()
df.head()

,title,link,description,cafename,cafeurl
0,전주 완산구 평화동 예비 초등 영어<b>과외</b> 삼천동 중등 국어<b>과외</b...,http://cafe.naver.com/kts9719/2792736,평화동생물1<b>과외</b> 평화동생물Ⅰ<b>과외</b> 평화동생물2<b>과외</b...,"닥공사★9급공무원,7급공무원,경찰,소...",https://cafe.naver.com/kts9719
1,전주 완산구 동서학동 예비 초등 영어<b>과외</b> 서서학동 중등 국어<b>과외<...,http://cafe.naver.com/kts9719/2792729,<b>과외</b> 동서학동생물Ⅰ<b>과외</b> 동서학동생물2<b>과외</b> 동서...,"닥공사★9급공무원,7급공무원,경찰,소...",https://cafe.naver.com/kts9719
2,전주 완산구 중앙동 예비 초등 영어<b>과외</b> 풍남동 중등 국어<b>과외</b...,http://cafe.naver.com/kts9719/2792725,중앙동생물1<b>과외</b> 중앙동생물Ⅰ<b>과외</b> 중앙동생물2<b>과외</b...,"닥공사★9급공무원,7급공무원,경찰,소...",https://cafe.naver.com/kts9719
3,안녕하세요^^가입인사드립니다.,http://cafe.naver.com/songdoeduall/36394,건전한 카페 문화를 위해 회원님들과 대화 시 존댓말을 사용해 <b>주세요</b>. ...,송도에듀올,https://cafe.naver.com/songdoeduall
4,초등 국어나 수학 <b>과외 추천해주세요</b>~,http://cafe.naver.com/pinklonos/62366,1:1 이였으면 좋겠어요 ~~,해남맘 우먼파워,https://cafe.naver.com/pinklonos


In [95]:
text = "전주 완산구 중앙동 예비 초등 영어<b>과외</b> 풍남동 중등 국어<b>과외</b>"
text2 = re.sub('[<b>|</b>]','',text)
text2

'전주 완산구 중앙동 예비 초등 영어과외 풍남동 중등 국어과외'

In [100]:
## title, description에 <b>,</b> 제거하기 
def clean_letter(x):
    x= re.sub('[<b>|</b>]','',x)
    return x

df['title'] = df['title'].apply(lambda x: re.sub('[<b>|</b>]',' ',x))
df['description'] = df['description'].apply(lambda x: re.sub('[<b>|</b>]',' ',x))

df.head()

,title,link,description,cafename,cafeurl
0,전주 완산구 평화동 예비 초등 영어 과외 삼천동 중등 국어 과외 ...,http://cafe.naver.com/kts9719/2792736,평화동생물1 과외 평화동생물Ⅰ 과외 평화동생물2 과외 ...,"닥공사★9급공무원,7급공무원,경찰,소...",https://cafe.naver.com/kts9719
1,전주 완산구 동서학동 예비 초등 영어 과외 서서학동 중등 국어 과외 ...,http://cafe.naver.com/kts9719/2792729,과외 동서학동생물Ⅰ 과외 동서학동생물2 과외 동서...,"닥공사★9급공무원,7급공무원,경찰,소...",https://cafe.naver.com/kts9719
2,전주 완산구 중앙동 예비 초등 영어 과외 풍남동 중등 국어 과외 ...,http://cafe.naver.com/kts9719/2792725,중앙동생물1 과외 중앙동생물Ⅰ 과외 중앙동생물2 과외 ...,"닥공사★9급공무원,7급공무원,경찰,소...",https://cafe.naver.com/kts9719
3,안녕하세요^^가입인사드립니다.,http://cafe.naver.com/songdoeduall/36394,건전한 카페 문화를 위해 회원님들과 대화 시 존댓말을 사용해 주세요 . ...,송도에듀올,https://cafe.naver.com/songdoeduall
4,초등 국어나 수학 과외 추천해주세요 ~,http://cafe.naver.com/pinklonos/62366,1:1 이였으면 좋겠어요 ~~,해남맘 우먼파워,https://cafe.naver.com/pinklonos


In [ ]:
## cafename에 지역구 이름 있는거 찾기-> 새로운컬럼 지역!

gu = ['강서','강동','강남','강북','송도','해남','광명','목동']

In [107]:
df.groupby('cafename').count()

,title,link,description,cafeurl
cafename,,,,
"!▷중정공☞ 중국유학생활,중국여행,무...",1,1,1,1
"""시크먼트"" 명품 정보 커뮤니티 - 샤넬...",3,3,3,3
"""예쁜카페 예카"" 예쁜그릇과앤틱로모노...",3,3,3,3
&lt;속초대표맘카페&gt;속대맘카페,1,1,1,1
&lt;오행공&gt; 오산맘들의 행복한 공간,1,1,1,1
...,...,...,...,...
홍천 엄마 사랑,1,1,1,1
화도사랑,1,1,1,1
황인영 영어 카페,18,18,18,18


## Test2..몇가지 카페에서 '과외추천' 게시글 검색하기?

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv 

def get_cafe_url(url):
    print(url)
    # webdriver 객체 생성
    driver = webdriver.Chrome('./chromedriver')

    # 페이지 접근
    # 날짜지정, 검색어지정, 50개씩보이기
    #url="https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26userDisplay=50%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=1"    #print(url)
    driver.get(url)
    driver.switch_to.frame('cafe_main')

    soup = bs(driver.page_source ,'html.parser')
    soup = soup.find_all(class_ = 'article-board m-tcol-c')[1]# 네이버 카페 구조 확인후 게시글 내용만 가저오기
    #datas = soup.find_all('td', class_ = 'td_article')

    datas_list=[]
    datas = soup.find_all(class_ = 'td_article')
    for data in datas:
        article_title = data.find(class_ = 'article')
        #link = article_title.get('href')
        article_title = article_title.get_text().strip()
        datas_list.append(article_title)
        #print(article_title)
        #print(baseurl + link)

        #f = open('cafe5.csv', 'a+', newline='', encoding = "euc-kr")# 문자 인코딩 -> euc-kr 형태로 변경하여 사용. 안되면 utf-8로 변경 후 진행
        #wr = csv.writer(f)
        #wr.writerow([article_title])
        #f.close()  
    return datas_list

In [15]:
import pandas as pd
from urllib.parse import urlencode,quote
from datetime import datetime

keyword = "과외추천"
#query=%B0%FA%BF%DC%C3%DF%C3%B5%26
today_date = datetime.today().strftime("%Y-%m-%d")

df_fin = pd.DataFrame()
for pg in range(1,45):
    url = "https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page="+str(pg)
    data_list = get_cafe_url(url)
    
    df = pd.DataFrame(data_list,columns=['article_title'])
    df_fin = pd.concat([df_fin,df],axis=0,ignore_index=True)

#df_fin.head()
filename = today_date+'-cafe_crawling.csv'  
df_fin.to_csv('./'+filename,encoding='utf-8-sig')

https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=1
https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=2
https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.i

https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=22
https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=23
https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search

https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=43
https://cafe.naver.com/hikicks?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=12946579%26search.media=0%26search.searchdate=2016-01-012021-09-13%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query=%B0%FA%BF%DC%C3%DF%C3%B5%26search.viewtype=title%26search.page=44
